In [1]:
import pandas as pd
from analytics_workbench.read_data import DataReader
from analytics_workbench.preprocessing import PreProcessor
from analytics_workbench.process_text import ProcessText
from analytics_workbench.unsupervised_learning import Unsupervised
from analytics_workbench.supervised_learning import ClassificationModelCV
from analytics_workbench.model_diagnostics import ClassificationModelDiagnotstics
from sklearn import metrics
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss,f1_score
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC,SVC
from sklearn.multiclass import OneVsRestClassifier
import dill
import pickle
import cPickle

In [2]:
df = pd.read_csv('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/input_dataframe')

In [4]:
df.head()

,domain,category,url_texts,page_text
0,11x17.com,Office_Products,Home Contact About Login or Register My Accoun...,.pr stars background image url http cdn.power...
1,a1components.com,Consumer_Electronics,Login My Account Help View Cart\n (0)\n...,Audio Video Tools Cables Speakers Networking ...
2,a1darts.com,Sporting_Goods,NaN,Error. Page cannot be displayed. Please conta...
3,a1-dogs.com,Pets,HOME DOG PRODUCTS Â Here's \n how...,dog products dog supplies and dog accessories...
4,a1florists.com,Flowers,Cities and Towns Business Cards Home\nPage Cit...,Canada s frendliest florists delivering flowe...


In [3]:
text_processor = ProcessText()
tfidf,vocabulary = text_processor.gen_dtm_from_files(df['page_text'],vectorizer_type='Tfidf',stem_type=None,
                                                     max_df=0.9,min_df=0.01,
                    n_gram_range=(1,1),stop_words_loc='/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/stopwords.txt')

tfidf.shape

KeyboardInterrupt: 

In [10]:
with open('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/page_text_tfidf.pkl','w') as f:
    dill.dump({'text_processor':text_processor,'tfidf':tfidf,'vocabulary':vocabulary},f)

In [12]:
text_processor = ProcessText()
tfidf,vocabulary = text_processor.gen_dtm_from_files(df['url_texts'],vectorizer_type='Tfidf',stem_type=None,
                                                     max_df=0.9,min_df=0.01,
                    n_gram_range=(1,1),stop_words_loc='/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/stopwords.txt')

tfidf.shape

(12594, 1592)

In [13]:
with open('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/url_texts_tfidf.pkl','w') as f:
    dill.dump({'text_processor':text_processor,'tfidf':tfidf,'vocabulary':vocabulary},f)

In [14]:
df.drop('page_text', axis=1, inplace=True)
df.drop('url_texts', axis=1, inplace=True)

#### classification model

In [3]:
with open('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/page_text_tfidf.pkl','r') as f:
    page_text_dic = pickle.load(f)

In [4]:
with open('/home/madan/Desktop/joswin_bck/toPendrive/works/company_classification/nlp-intelligence/e-commerce_classification/alexa_based/url_texts_tfidf.pkl','r') as f:
    url_text_dic = pickle.load(f)

In [19]:
from scipy.sparse import hstack

In [38]:
tfidf = hstack([page_text_dic['tfidf'],url_text_dic['tfidf']]).tocsr()

In [13]:
df.loc[df['category'].isin(['Holidays_and_Celebrations','Beauty_Products','Crafts']),'category'] = 'other_categories_1'
df.loc[df['category'].isin(['Auctions', 'Publications', 'Computers', 'Consumer_Information', 'Death_Care', 'Health',
                          'Travel_Items', 'Sports', 'Toys_and_Games', 'Ethnic_and_Regional', 'Tools', 'Niche', 
                          'Weddings', 'Holidays', 'Tobacco']),'category'] = 'other_categories_2'
df['category'].value_counts()

Home_and_Garden              1568
Clothing                     1136
other_categories_2           1063
Sporting_Goods               1040
Food                          773
Recreation                    684
Antiques_and_Collectibles     641
Consumer_Electronics          640
Pets                          494
Jewelry                       492
Music                         433
Entertainment                 405
Health_and_Healthcare         380
Books                         363
General_Merchandise           353
Gifts                         331
Children                      330
other_categories_1            315
Other                         258
Flowers                       258
Office_Products               252
Photography                   218
Classifieds                   167
Name: category, dtype: int64

In [33]:
lbl = preprocessing.LabelEncoder()
categories_transformed = lbl.fit_transform(df['category'].values)

In [28]:
predictions = np.zeros(categories_transformed.shape)

cv_f1_scores,cv_log_loss = [],[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(tfidf.shape[0])):
        dev_X, val_X = tfidf[dev_index,:], tfidf[val_index,:]
        dev_y, val_y = categories_transformed[dev_index], categories_transformed[val_index]
        clf = LinearSVC(C=0.2)
        clf.fit(dev_X,dev_y)
        preds_labels = clf.predict(val_X)
        predictions[val_index] = preds_labels
        cv_f1_scores.append(f1_score(val_y, preds_labels,average='weighted'))
        #cv_log_loss.append(log_loss(val_y,preds_labels))
        print(cv_f1_scores)
        #print(cv_log_loss)

[0.51039969821862263]
[0.51039969821862263, 0.52724863785420617]
[0.51039969821862263, 0.52724863785420617, 0.52139352745904977]
[0.51039969821862263, 0.52724863785420617, 0.52139352745904977, 0.51546986185172594]
[0.51039969821862263, 0.52724863785420617, 0.52139352745904977, 0.51546986185172594, 0.52283009923841939]


In [29]:
print metrics.classification_report(categories_transformed,predictions,labels=range(23),target_names=list(lbl.classes_))
pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

                           precision    recall  f1-score   support

Antiques_and_Collectibles       0.47      0.39      0.43       641
                 Auctions       0.41      0.09      0.15        77
          Beauty_Products       0.66      0.60      0.63       194
                    Books       0.52      0.42      0.46       363
                 Children       0.65      0.45      0.53       330
              Classifieds       0.64      0.69      0.67       167
                 Clothing       0.44      0.63      0.52      1136
                Computers       0.57      0.56      0.57       500
     Consumer_Electronics       0.64      0.55      0.59       640
     Consumer_Information       0.27      0.06      0.09       106
                   Crafts       0.00      0.00      0.00         6
               Death_Care       0.00      0.00      0.00         9
            Entertainment       0.53      0.44      0.48       405
      Ethnic_and_Regional       0.00      0.00      0.00     

,Antiques_and_Collectibles,Auctions,Beauty_Products,Books,Children,Classifieds,Clothing,Computers,Consumer_Electronics,Consumer_Information,...,Photography,Publications,Recreation,Sporting_Goods,Sports,Tobacco,Tools,Toys_and_Games,Travel_Items,Weddings
Antiques_and_Collectibles,251,0,1,13,1,9,48,4,8,1,...,5,6,27,17,0,0,0,1,0,0
Auctions,16,7,2,2,0,7,2,4,2,0,...,0,0,2,7,0,0,0,0,0,0
Beauty_Products,1,1,117,0,1,0,17,2,0,1,...,0,0,0,2,0,0,0,1,0,0
Books,21,0,2,151,5,4,28,3,6,2,...,0,0,12,15,0,0,0,0,0,0
Children,4,0,1,4,150,1,41,3,1,1,...,3,0,2,10,0,0,0,0,0,0
Classifieds,6,0,0,2,0,116,7,4,1,0,...,0,0,3,2,0,0,0,0,0,0
Clothing,16,2,6,6,21,2,718,26,8,1,...,4,1,16,44,0,0,0,0,0,0
Computers,4,0,3,2,0,2,35,282,34,0,...,7,0,5,18,0,0,1,0,0,0
Consumer_Electronics,14,0,1,9,0,2,54,28,352,1,...,13,1,7,17,0,0,0,1,0,0
Consumer_Information,2,0,0,4,3,3,19,1,15,6,...,4,2,1,3,0,0,0,0,0,0


In [43]:
predictions = np.zeros(categories_transformed.shape)

cv_f1_scores,cv_log_loss = [],[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(tfidf.shape[0])):
        dev_X, val_X = tfidf[dev_index,:], tfidf[val_index,:]
        dev_y, val_y = categories_transformed[dev_index], categories_transformed[val_index]
        clf = MultinomialNB(fit_prior=False)
        clf.fit(dev_X,dev_y)
        preds_labels = clf.predict(val_X)
        predictions[val_index] = preds_labels
        cv_f1_scores.append(f1_score(val_y, preds_labels,average='weighted'))
        #cv_log_loss.append(log_loss(val_y,preds_labels))
        print(cv_f1_scores)
        #print(cv_log_loss)

[0.51398283339070661]
[0.51398283339070661, 0.51441031798901704]
[0.51398283339070661, 0.51441031798901704, 0.52891723328958573]
[0.51398283339070661, 0.51441031798901704, 0.52891723328958573, 0.5156568326440889]
[0.51398283339070661, 0.51441031798901704, 0.52891723328958573, 0.5156568326440889, 0.52130871052896199]


In [44]:
print metrics.classification_report(categories_transformed,predictions,labels=range(23),target_names=list(lbl.classes_))
pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

                           precision    recall  f1-score   support

Antiques_and_Collectibles       0.46      0.52      0.49       641
                    Books       0.61      0.31      0.41       363
                 Children       0.77      0.29      0.42       330
              Classifieds       0.87      0.52      0.65       167
                 Clothing       0.42      0.74      0.53      1136
     Consumer_Electronics       0.66      0.56      0.61       640
            Entertainment       0.68      0.37      0.48       405
                  Flowers       0.89      0.72      0.80       258
                     Food       0.60      0.79      0.69       773
      General_Merchandise       0.35      0.03      0.05       353
                    Gifts       0.56      0.46      0.50       331
    Health_and_Healthcare       0.76      0.28      0.40       380
          Home_and_Garden       0.47      0.72      0.56      1568
                  Jewelry       0.78      0.72      0.75     

,Antiques_and_Collectibles,Books,Children,Classifieds,Clothing,Consumer_Electronics,Entertainment,Flowers,Food,General_Merchandise,...,Jewelry,Music,Office_Products,Other,Pets,Photography,Recreation,Sporting_Goods,other_categories_1,other_categories_2
Antiques_and_Collectibles,336,2,0,3,70,4,5,1,17,0,...,16,2,6,3,0,2,23,17,3,50
Books,29,114,1,2,28,4,8,0,14,1,...,1,2,0,0,5,0,5,10,2,102
Children,4,3,96,0,67,1,2,0,9,1,...,7,2,1,0,1,0,1,12,0,34
Classifieds,6,0,1,87,8,0,0,2,2,0,...,0,0,0,0,1,0,9,5,0,38
Clothing,18,4,6,3,839,2,4,0,14,0,...,7,1,4,0,1,0,6,39,2,120
Consumer_Electronics,20,3,0,0,67,361,3,0,1,2,...,1,4,0,0,2,5,4,10,3,101
Entertainment,12,11,1,0,44,12,150,1,11,0,...,0,23,3,0,2,0,4,18,3,66
Flowers,3,1,0,0,8,1,1,187,20,0,...,0,1,0,0,0,0,0,1,1,7
Food,11,1,0,0,41,1,3,4,613,0,...,0,0,0,0,1,0,2,6,1,28
General_Merchandise,21,1,3,1,59,6,2,2,21,9,...,16,1,0,1,5,0,8,14,7,89


In [14]:
tfidf = url_text_dic['tfidf']
lbl = preprocessing.LabelEncoder()
categories_transformed = lbl.fit_transform(df['category'].values)
# analysing using only links
predictions = np.zeros(categories_transformed.shape)

cv_f1_scores,cv_log_loss = [],[]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(tfidf.shape[0])):
    dev_X, val_X = tfidf[dev_index,:], tfidf[val_index,:]
    dev_y, val_y = categories_transformed[dev_index], categories_transformed[val_index]
    clf = MultinomialNB(fit_prior=False)
    clf.fit(dev_X,dev_y)
    preds_labels = clf.predict(val_X)
    predictions[val_index] = preds_labels
    cv_f1_scores.append(f1_score(val_y, preds_labels,average='weighted'))
    #cv_log_loss.append(log_loss(val_y,preds_labels))
    print(cv_f1_scores)
    #print(cv_log_loss)
print metrics.classification_report(categories_transformed,predictions,labels=range(23),target_names=list(lbl.classes_))
pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

[0.4998051078801925]
[0.4998051078801925, 0.50785739976946398]
[0.4998051078801925, 0.50785739976946398, 0.517651441632208]
[0.4998051078801925, 0.50785739976946398, 0.517651441632208, 0.49937255194789115]
[0.4998051078801925, 0.50785739976946398, 0.517651441632208, 0.49937255194789115, 0.50873131807956207]
                           precision    recall  f1-score   support

Antiques_and_Collectibles       0.20      0.55      0.29       641
                    Books       0.42      0.46      0.44       363
                 Children       0.57      0.47      0.51       330
              Classifieds       0.50      0.74      0.59       167
                 Clothing       0.50      0.57      0.54      1136
     Consumer_Electronics       0.58      0.56      0.57       640
            Entertainment       0.54      0.43      0.48       405
                  Flowers       0.69      0.81      0.75       258
                     Food       0.53      0.73      0.61       773
      General_Mercha

,Antiques_and_Collectibles,Books,Children,Classifieds,Clothing,Consumer_Electronics,Entertainment,Flowers,Food,General_Merchandise,...,Jewelry,Music,Office_Products,Other,Pets,Photography,Recreation,Sporting_Goods,other_categories_1,other_categories_2
Antiques_and_Collectibles,353,21,1,16,33,5,13,2,29,2,...,25,5,12,8,6,5,21,8,2,29
Books,50,168,6,7,10,6,11,0,19,2,...,1,9,3,1,7,0,3,3,2,39
Children,28,6,154,1,25,1,6,2,16,3,...,8,5,2,0,5,2,1,6,2,19
Classifieds,15,3,0,123,3,0,0,2,4,0,...,0,0,1,0,1,0,2,0,0,10
Clothing,128,17,23,15,651,9,13,5,37,8,...,13,4,9,9,7,5,7,27,9,100
Consumer_Electronics,80,8,2,3,37,357,5,1,9,1,...,2,6,1,0,1,8,5,9,2,80
Entertainment,51,17,1,6,17,16,174,2,15,0,...,1,22,1,2,3,3,6,5,8,31
Flowers,17,2,1,3,2,0,0,209,8,0,...,0,0,0,0,1,1,0,0,0,4
Food,75,8,0,3,23,3,3,10,566,3,...,1,3,1,2,7,0,2,5,2,28
General_Merchandise,52,7,9,5,37,13,3,3,27,35,...,20,3,1,4,8,1,6,6,11,49


In [15]:
tmp = pd.DataFrame(metrics.confusion_matrix(categories_transformed,predictions),columns=list(lbl.classes_),index=list(lbl.classes_))

In [18]:
tmp.ix['Home_and_Garden',:]

Antiques_and_Collectibles    206
Books                         14
Children                      20
Classifieds                   10
Clothing                      61
Consumer_Electronics          36
Entertainment                  8
Flowers                       31
Food                         109
General_Merchandise            7
Gifts                         33
Health_and_Healthcare         15
Home_and_Garden              750
Jewelry                       20
Music                          9
Office_Products               19
Other                          7
Pets                          33
Photography                   12
Recreation                    24
Sporting_Goods                21
other_categories_1            14
other_categories_2           109
Name: Home_and_Garden, dtype: int64